In [1]:
import numpy as np
import cv2
# Based on : https://gist.github.com/soruly/bd02a218690fe4e19295de3f5bede242
def compare(img1_path, img2_path):
    # Read in images
    img1, img2 = cv2.imread(img1_path), cv2.imread(img2_path)                
    # https://stackoverflow.com/questions/65541488/dead-kernel-sift-detectgray-none-the-kernel-appears-to-have-died-it-will-re
    # Using cv2.SIFT_create() to avoid kernel dying, like it does with cv2.SIFT()
    sift = cv2.SIFT_create() # Initializing SIFT

    # Detect keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Match features between the two images
    bf = cv2.BFMatcher()
    matches = bf.match(des1,des2)
    return matches

In [2]:
matches = compare('./data/test/dog1.jpg', './data/test/dog2.jpg')
len(matches)

1754

In [3]:
matches = compare('./data/test/cat1.jpg', './data/test/dog2.jpg')
len(matches)

106303

In [4]:
matches = compare('./data/test/cat1.jpg', './data/test/dog1.jpg')
len(matches)

106303